In [1]:
from Preprocessor import Preprocessor

#FilterReview
from sklearn.model_selection import train_test_split 
from sklearn.naive_bayes import GaussianNB 
from sklearn import metrics 
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import pickle 
import os.path

'''
readCSV :CSV 파일을 읽는다. 고정포맷이 정해짐 
trainModel(ts =0.25): 입력된 모델에서 학습시킨다. ts 테스트 케이스 
predict(): 학습시킨 모델에 test데이터가 입력되면 예측한다. 
writePredictionResult(): 예측 결과를 파일로 출력한다. 
'''
class FilterReviewSV:
    def __init__(self):
        
        self.model_filename = 'model/FilterReviewSV_model'
        self.isSetModel = False
        if not(self.isSetModel) and os.path.isfile(self.model_filename+'.sav'):
            print('모델을 불러오는중...')
            self.gnb = pickle.load(open(self.model_filename+'.sav', 'rb'))
            
    def input_X(self, para):
        pre = Preprocessor()
        #print(para)
        pre.setPara(para)
        #print(pre.sentences)
        self.X = pre.transVector()
    def input_Y(self, result):
        self.Y = result
        
    def readCSV(self,filename,tag=True):
        self.data = pd.read_csv(filename,engine='python')
        self.X_content = self.data['sentence'].values
        self.input_X(self.X_content)
        if tag == True:
            self.input_Y(self.data['label'].values)
        
    def trainModel(self,ts):
        self.X_train,self.X_test,self.y_train,self.y_test=train_test_split(self.X,self.Y,test_size=ts)
        self.gnb = GaussianNB()
        self.y_pred = self.gnb.fit(self.X_train, self.y_train).predict(self.X_test)

        # 테스트 socre 확인
        print("Number of right labeled points out of a total %d points : %.2f %%"
      % (len(self.y_test),100*(len(self.y_test)-(self.y_test != self.y_pred).sum())/len(self.y_test)))
        
        pickle.dump(self.gnb, open(self.model_filename+'.sav', 'wb'))
        isSetModel = True
    def predict(self):
        self.y_pred = self.gnb.predict(self.X)
        self.df = pd.DataFrame({'pr_label':self.y_pred, 'sentence':self.X_content})
        return self.y_pred
    def writePredictionResult(self, filename):
        self.df.to_csv(filename,encoding='utf-8-sig',index=False)
        


C:\Anaconda3\envs\konlpy\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import glob
import os

#fr.readCSV(tag=False,filename='Data_ReviewFilter/TrainData.csv')
#print('\\n Readed\\n')
#fr.trainModel(ts=0.25)

folderlist = glob.glob('./data2/*')
for fol in folderlist:
    if os.path.isdir(fol):
        print('folder:',fol)
      
        filelist = glob.glob(fol+'/*.csv')
        for file in filelist:
            input_file = file.replace('\\\\','/')
            input_path = input_file[2:]
            print('input_file: ',input_path)
            print('\\n Readed\\n')
    
            fr = FilterReviewSV()
            fr.readCSV(tag=False,filename=input_path)
            fr.predict()
            output_file= 'labeled_'+input_path[11:]
            fr.writePredictionResult(output_file)

folder: ./data2\2015
input_file:  data2\2015\오르비_content_수능특강_화학 _2015.01.01. ~ 2015.12.31._split.csv
\n Readed\n
모델을 불러오는중...


C:\Anaconda3\envs\konlpy\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator GaussianNB from version 0.19.1 when using version 0.19.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


folder: ./data2\2016
input_file:  data2\2016\오르비_content_수능특강_화학 _2016-01-01 ~ 2016-12-31_split.csv
\n Readed\n
모델을 불러오는중...
folder: ./data2\2017
input_file:  data2\2017\오르비_content_수능특강_화학 _2017-01-01 ~ 2017-12-31_split.csv
\n Readed\n
모델을 불러오는중...
